In [20]:
import pandas as pd
import os
import pickle
import sys
import importlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import optuna

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error



In [3]:

package_name = "mlflow"
try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ModuleNotFoundError:
    print(f"{package_name} not found. Installing...")
    %pip install {package_name}

mlflow is already installed.


In [4]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week1/data/yellow_tripdata_2021-02.parquet')

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [9]:

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [11]:
X_train

<1343254x27306 sparse matrix of type '<class 'numpy.float64'>'
	with 2686508 stored elements in Compressed Sparse Row format>

In [14]:
def get_size_mb(variable):
    size_in_bytes = sys.getsizeof(variable)
    size_in_mb = size_in_bytes / (1024 * 1024)
    return size_in_mb

In [15]:
size_my_list = get_size_mb(train_dicts)
size_my_dict = get_size_mb(X_train)

In [16]:
print(f"Size of 'my_list': {size_my_list:.2f} MB")
print(f"Size of 'my_dict': {size_my_dict:.2f} MB")

Size of 'my_list': 11.47 MB
Size of 'my_dict': 0.00 MB


In [17]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [18]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-optuna-randomforest")

2023/04/15 15:19:17 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/15 15:19:17 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/04/15 15:19:17 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-optuna-randomforest' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/bcthakreda/mlops_zoomcamp/Machine-Learning-ZoomCamp/week2/mlruns/6', creation_time=1681571957410, experiment_id='6', last_update_time=1681571957410, lifecycle_stage='active', name='nyc-taxi-optuna-randomforest', tags={}>